In [2]:
from langgraph.graph import StateGraph, END, START
from langchain_openai import ChatOpenAI
import os
from typing import Optional
from dotenv import load_dotenv
from IPython.display import  Image, display
from typing import Dict, Any
from langchain.tools import StructuredTool
from langgraph.prebuilt import ToolNode, create_react_agent
from langchain_core.messages import AIMessage, ToolMessage

In [14]:
load_dotenv()
openai_api_key=os.getenv('OPENAI_API_KEY')
llm = ChatOpenAI(model="gpt-4.1-mini-2025-04-14", temperature=0)

In [4]:
def get_investment_assets() -> Dict[str, Any]:
    assets = [
        {
            "nome": "Tesouro Selic 2029",
            "descricao": "Título público pós-fixado atrelado à taxa Selic. Alta liquidez, baixo risco, indicado para reserva de emergência.",
            "rentabilidade": "Selic + 0.10% ao ano",
            "isento_ir": False,
            "liquidez": "Diária",
            "prazo": "Vencimento em 2029",
            "risco": "Baixíssimo (soberano)"
        },
        {
            "nome": "CDB Banco Itaúvest",
            "descricao": "Certificado de Depósito Bancário emitido por banco médio. Protegido pelo FGC até R$ 250 mil por CPF e instituição.",
            "rentabilidade": "120% do CDI",
            "isento_ir": False,
            "liquidez": "Após 6 meses",
            "prazo": "3 anos",
            "risco": "Médio (dependente do emissor)"
        },
        {
            "nome": "FII XPML11",
            "descricao": "Fundo Imobiliário que investe em lajes corporativas de alto padrão em São Paulo.",
            "rentabilidade": "Dividend Yield médio de 8% a.a.",
            "isento_ir": True,
            "liquidez": "Negociação em bolsa (B3)",
            "prazo": "Indeterminado",
            "risco": "Médio (vacância, preços de imóveis)"
        }
    ]
    return {"ativos": assets}

# Criando a Tool
investment_tool = StructuredTool.from_function(
    func=get_investment_assets,
    name="investment_assets_tool",
    description=(
        """Retorna informações detalhadas sobre três ativos de investimento específicos: Tesouro Selic 2029, FII XPML11 e 
        CDB Banco Itaúvest. Os dados de cada ativo inclui descrição, rentabilidade, isenção de IR, liquidez, prazo e risco."""
    )
)


In [5]:
investment_prompt = """
Você é um agente financeiro especializado em responder perguntas sobre investimentos.

Instruções:
1. Se a pergunta envolver **ativos de investimento específicos** (Tesouro Selic, CDB, FII), utilize a tool `investment_assets_tool`.
2. Se a pergunta envolver **conceitos de investimento** (como Taxa Selic, IPCA ou Bolsa de Valores do Brasil), utilize apenas as informações 
abaixo nos conceitos de investimentos.
3. Caso a pergunta não se enquadre em nenhum dos dois casos acima, utilize o conhecimento base do LLM.

Conceitos de Investimento:

- **Taxa Selic**:  
  É a taxa básica de juros da economia brasileira, definida pelo Banco Central. Serve como referência para empréstimos, financiamentos e aplicações de renda fixa. Quando a Selic sobe, o crédito fica mais caro e a rentabilidade de títulos pós-fixados aumenta.

- **IPCA (Índice Nacional de Preços ao Consumidor Amplo)**:  
  É o principal índice de inflação do Brasil, calculado pelo IBGE. Mede a variação de preços de bens e serviços consumidos pelas famílias. Serve como referência para reajustes de contratos, metas do governo e indexação de títulos públicos como o Tesouro IPCA+.

- **Bolsa de Valores do Brasil (B3)**:  
  É o mercado organizado onde são negociadas ações, fundos imobiliários e derivativos. Permite que empresas captem recursos e investidores se tornem sócios. O principal índice é o Ibovespa, que reflete o desempenho médio das ações mais negociadas.
"""


In [6]:
# Define o agente com tool
agent = create_react_agent(
    model=llm,
    tools=[investment_tool],
    name="assistant",
    prompt=investment_prompt
)

In [11]:
question = "O que é xpml11?"
msgs = {"messages": [{"role": "user", "content": question}]}

In [ ]:
response = agent.invoke(msgs)
print(response)

{'messages': [HumanMessage(content='O que é xpml11?', additional_kwargs={}, response_metadata={}, id='1d7ef7df-0b08-408f-833a-38c0f6858c10'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_SDeNecOJmd8Kd6hPJiI4O78p', 'function': {'arguments': '{}', 'name': 'investment_assets_tool'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 424, 'total_tokens': 437, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_3502f4eb73', 'id': 'chatcmpl-CKU2OTVRFuLlO6uIvhhzuCTePprlc', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='assistant', id='run--2aaaaaac-fb80-495f-9493-0e1048eed1c9-0', tool_calls=[{'name': 'investment_assets_tool', 'args': {}, 'id': 'call_SDeNecO

In [ ]:
for msg in response["messages"]:
    if isinstance(msg, AIMessage) and msg.content.strip():
        print("🧠 Resposta do agente:", msg.content)
    elif isinstance(msg, ToolMessage) and msg.content.strip():
        print("🔧 Retorno da tool:", msg.content)

🔧 Retorno da tool: {"ativos": [{"nome": "Tesouro Selic 2029", "descricao": "Título público pós-fixado atrelado à taxa Selic. Alta liquidez, baixo risco, indicado para reserva de emergência.", "rentabilidade": "Selic + 0.10% ao ano", "isento_ir": false, "liquidez": "Diária", "prazo": "Vencimento em 2029", "risco": "Baixíssimo (soberano)"}, {"nome": "CDB Banco Itaúvest", "descricao": "Certificado de Depósito Bancário emitido por banco médio. Protegido pelo FGC até R$ 250 mil por CPF e instituição.", "rentabilidade": "120% do CDI", "isento_ir": false, "liquidez": "Após 6 meses", "prazo": "3 anos", "risco": "Médio (dependente do emissor)"}, {"nome": "FII XPML11", "descricao": "Fundo Imobiliário que investe em lajes corporativas de alto padrão em São Paulo.", "rentabilidade": "Dividend Yield médio de 8% a.a.", "isento_ir": true, "liquidez": "Negociação em bolsa (B3)", "prazo": "Indeterminado", "risco": "Médio (vacância, preços de imóveis)"}]}
🧠 Resposta do agente: XPML11 é um Fundo de Inves

In [15]:
for chunk in agent.stream(msgs):
    # Pega o conteúdo do agente
    if 'agent' in chunk:
        for message in chunk['agent']['messages']:
            if isinstance(message, AIMessage) and message.content.strip():
                print("🧠 Resposta do agente:", message.content)

    # Pega o conteúdo da tool (resposta bruta da função)
    if 'tools' in chunk:
        for message in chunk['tools']['messages']:
            if isinstance(message, ToolMessage) and message.content.strip():
                print("🔧 Resposta da tool:", message.content)


🔧 Resposta da tool: {"ativos": [{"nome": "Tesouro Selic 2029", "descricao": "Título público pós-fixado atrelado à taxa Selic. Alta liquidez, baixo risco, indicado para reserva de emergência.", "rentabilidade": "Selic + 0.10% ao ano", "isento_ir": false, "liquidez": "Diária", "prazo": "Vencimento em 2029", "risco": "Baixíssimo (soberano)"}, {"nome": "CDB Banco Itaúvest", "descricao": "Certificado de Depósito Bancário emitido por banco médio. Protegido pelo FGC até R$ 250 mil por CPF e instituição.", "rentabilidade": "120% do CDI", "isento_ir": false, "liquidez": "Após 6 meses", "prazo": "3 anos", "risco": "Médio (dependente do emissor)"}, {"nome": "FII XPML11", "descricao": "Fundo Imobiliário que investe em lajes corporativas de alto padrão em São Paulo.", "rentabilidade": "Dividend Yield médio de 8% a.a.", "isento_ir": true, "liquidez": "Negociação em bolsa (B3)", "prazo": "Indeterminado", "risco": "Médio (vacância, preços de imóveis)"}]}
🧠 Resposta do agente: XPML11 é um Fundo de Inve

{'messages': [HumanMessage(content='O que é xpml11?', additional_kwargs={}, response_metadata={}, id='d7cef59b-375d-4fbb-88d6-50c049a266cf'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_qltlH2SLV43tCoNSGDDP1O5F', 'function': {'arguments': '{}', 'name': 'investment_assets_tool'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 424, 'total_tokens': 437, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_3502f4eb73', 'id': 'chatcmpl-CKTWtOvk9vt5UPpqE8BlTmaHu96QX', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='assistant', id='run--5b46675b-3bb2-4bac-8e49-41197d8aa366-0', tool_calls=[{'name': 'investment_assets_tool', 'args': {}, 'id': 'call_qltlH

In [ ]:
for chunk in agent.stream(msgs):
    print(chunk)

In [28]:
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

In [30]:
import nest_asyncio

In [32]:
nest_asyncio.apply()
display(
    Image(
        agent.get_graph().draw_mermaid_png(
            curve_style=CurveStyle.LINEAR,
            node_colors=NodeStyles(first="#ffdfba", last="#baffc9", default="#fad7de"),
            wrap_label_n_words=9,
            output_file_path=None,
            draw_method=MermaidDrawMethod.PYPPETEER,
            background_color="white",
            padding=10,
        )
    )
)

[INFO] Starting Chromium download.


OSError: Chromium downloadable not found at https://storage.googleapis.com/chromium-browser-snapshots/Win_x64/1181205/chrome-win.zip: Received <?xml version='1.0' encoding='UTF-8'?><Error><Code>NoSuchKey</Code><Message>The specified key does not exist.</Message><Details>No such object: chromium-browser-snapshots/Win_x64/1181205/chrome-win.zip</Details></Error>.


In [26]:
display(Image(agent.get_graph().draw_png()))

ImportError: Install pygraphviz to draw graphs: `pip install pygraphviz`.

In [ ]:
# Exemplo de execução
query = "O que é IPCA?"
for s in app.stream({"messages": [("user", query)]}):
    print(s)